### 1) El servicio meteorológico registra datos del tiempo para todas las ciudades donde posee una base de medición. Esta información se encuentra en dos RDD. En el primero se tiene información de las bases de medición: 

###(ID_BASE, NOMBRE, PCIA, CIUDAD, LAT, LON).
 
###El segundo posee información sobre las mediciones en sí: 
###(TIMESTAMP, ID_BASE, TEMPERATURA, HUMEDAD, PRESIÓN, DIRECCIÓN VIENTO, VELOCIDAD VIENTO).

###Se desea obtener un reporte para las bases de la Provincia de Buenos Aires. El mismo debeinformar los ID y nombre de bases de medición que hayan registrado una variación de temperatura promedio mensual mayor al 30% en el año 2018 (dada la temperatura promedio de un mes, esta se debe comparar con el promedio del mes anterior, para determinar la variación porcentual). Resolver utilizando el API de RDD de PySpark, dando el reporte en un RDD.

In [ ]:
#Filtro las bases de la provincia de Buenos Aires
Bases = BasesRDD.filter(lambda x: x[3] == 'Buenos Aires')

In [ ]:
#Ordeno mi RDD Bases eliminando campos innecesarios y lo preparo para un join
#(id_base, nombre)
Bases = Bases.map(lambda x: (x[0], x[1]))

In [ ]:
#Ordeno mi RDD Mediciones y lo preparo para el join
#(id_base, fecha, temperatura)
Mediciones = MedicionesRDD.map(lambda x: (x[1], x[0], x[2] ))

In [ ]:
#Junto ambos RDD para obtener las mediciones unicamente de Buenos Aires
#(id_base, (nombre, fecha, temperatura) )
MedicionesBsAs = Bases.LeftOuterJoin(Mediciones)

In [ ]:
#Filtro por año 2018, porque ahora tengo las mediciones de Buenos Aires pero de todos los años
d1 = datetime.strptime('2017-12-01' , '%YYYY-%MM-%DD')
d2 = datetime.strptime('2018-12-31' , '%YYYY-%MM-%DD')
MedicionesBsAs2018 = MedicionesBsAs.filter(lambda x: d2 > x[1][1] > d1 )

In [ ]:
#Organizo mi RDD para que quede de la forma ( (base,año,mes), (temperatura, 1, nombre) )
MedicionesBsAs2018 = MedicionesBsAs2018.map(lambda x: ((x[0], x[1][1].year, x[1][1].month)), (x[1][2], 1, x[1][0]) )

In [ ]:
#Calculo el promedio mensual, para esto tengo que agrupar por (base,año,mes)
#Obtengo ( (base,año,mes), (temperatura, 1, nombre) )
MedicionesBsAs2018 = MedicionesBsAs2018.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1], a[0]))

In [ ]:
#Reordeno mi RDD para calcular el promedio ((base,nombre) , ((año,mes),promedio_mensual) )
MedicionesBsAs2018 = MedicionesBsAs2018.map(lambda x: ((x[0][0], x[1][2]), ((x[0][1], x[0][2]), x[1][0]/x[1][1])))

In [ ]:
#Agrupo por base y obtengo todos los promedios de todos los meses en una lista
#((base, nombre), [((año,mes), temp_promedio)), ..., ...., ... ]
MedicionesBsAs2018 = MedicionesBsAs2018.groupByKey()